# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/wXfah/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298264725400                   -0.85    5.0         
  2   -8.300170955052       -2.72       -1.25    1.1    122ms
  3   -8.300431835255       -3.58       -1.89    3.1    312ms
  4   -8.300461414057       -4.53       -2.77    2.4    133ms
  5   -8.300464234011       -5.55       -3.07    3.1    161ms
  6   -8.300464436400       -6.69       -3.25    8.2    220ms
  7   -8.300464535820       -7.00       -3.40    2.5    151ms
  8   -8.300464593464       -7.24       -3.56    1.4    136ms
  9   -8.300464628246       -7.46       -3.74    1.5    167ms
 10   -8.300464635423       -8.14       -3.85    1.4    118ms
 11   -8.300464642898       -8.13       -4.14    2.1    138ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.63708438424                   -0.71    5.2         
  2   -16.64116490828       -2.39       -1.15    1.0    272ms
  3   -16.67915946460       -1.42       -1.75    3.2    333ms
  4   -16.67927170394       -3.95       -2.37    4.9    438ms
  5   -16.67928419121       -4.90       -2.94    3.1    349ms
  6   -16.67928587677       -5.77       -3.27    4.1    397ms
  7   -16.67928607117       -6.71       -3.41    1.5    285ms
  8   -16.67928621947       -6.83       -4.29    1.1    263ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32764966630                   -0.56    7.5         
  2   -33.33508252425       -2.13       -1.00    1.2    1.13s
  3   -33.33674772818       -2.78       -1.71    5.6    1.58s
  4   -33.33691681886       -3.77       -2.59    3.0    1.27s
  5   -33.33694293523       -4.58       -3.18    6.8    1.77s
  6   -33.33694370906       -6.11       -3.93    5.1    1.50s
  7   -33.33694378143       -7.14       -4.54    4.0    1.57s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298354872601                   -0.85    5.1         
  2   -8.300249380374       -2.72       -1.59    1.0   88.9ms
  3   -8.300429968128       -3.74       -2.62    3.1    125ms
  4   -8.300413256689   +   -4.78       -2.40    5.4    181ms
  5   -8.300463827627       -4.30       -3.27    1.0   92.8ms
  6   -8.300464572925       -6.13       -3.70    3.5    149ms
  7   -8.300464638330       -7.18       -4.30    1.1    136ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32387873147                   -0.56    6.5         
  2   -33.32304112683   +   -3.08       -1.27    1.6    956ms
  3   -20.98954541109   +    1.09       -0.52    5.8    1.77s
  4   -33.28620394548        1.09       -1.63    6.6    1.89s
  5   -33.25077854527   +   -1.45       -1.32    2.8    1.31s
  6   -33.18330364387   +   -1.17       -1.47    3.2    1.35s
  7   -33.25170834535       -1.16       -1.59    3.9    1.30s
  8   -33.33592598553       -1.07       -2.44    2.8    1.12s
  9   -33.33643668077       -3.29       -2.66    4.4    1.50s
 10   -33.33676757745       -3.48       -2.61    3.0    1.22s
 11   -33.33687816723       -3.96       -3.04    1.9    1.08s
 12   -33.33693147222       -4.27      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.